In [1]:
%matplotlib notebook
import numpy as np
import operator
import scipy
import scipy.ndimage
import matplotlib.patches
from scipy.spatial import distance
import photutils
from photutils import centroid_com
import h5py
import matplotlib
import matplotlib.pyplot as mpl
from matplotlib import pyplot as plt
import sys
import os
import os.path
import cv2
from scipy.ndimage.morphology import binary_fill_holes

In [3]:
files = {'r0144':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0144_type1-peaks.h5',
         'r0145':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0145_type1-peaks.h5',
         'r0146':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0146_type1-peaks.h5',
         'r0151':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0151_type1-peaks.h5',
         'r0165':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0165_type1-peaks.h5',
         'r0166':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0166_type1-peaks.h5',
         'r0167':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0167_type1-peaks.h5',
         'r0168':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0168_type1-peaks.h5',
         'r0169':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0169_type1-peaks.h5',
         'r0170':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0170_type1-peaks.h5',
         'r0171':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0171_type1-peaks.h5',
         'r0172':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0172_type1-peaks.h5',
         'r0173':'/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/r0173_type1-peaks.h5'}


f_tuples = sorted(files.items(), key=operator.itemgetter(1))
runs = sorted(np.array([n[0] for n in f_tuples]))
files_pass = np.array([n[1] for n in sorted(f_tuples)])

### wavelength should be corrected in the files, they are not in Angstrom

In [55]:
def load_runs(run,path=None):
    '''
    loads data from processed h5
    '''
    
    if path is None:
        h5path = '/Users/niloo/Desktop/LCLS/14Dec2020/LCLS_files/%s_type1-peaks.h5'%run
    else:
        h5path = path+'%s_type1-peaks.h5'%run
    h5file = h5py.File(h5path,'r')
    wavelength = h5file['LCLS/wavelengthInAngstromMean'][()]/1e10
    n_shots = h5file['LCLS/numberOfShots'][()]
    detector_distance = h5file['LCLS/detectorDistanceInMeterMean'][()]
    n_peaks = []
    peak_list = []
    center_mass = []
    shots = [x for x in h5file['shots'].keys()]
    for i,j in enumerate(shots):
        n_peaks.append(h5file['shots'][j]['numberOfPeaks'][()])
        each_shot = [key for key in h5file['shots'][j].keys()]
        for i in each_shot:
            if (len(i) == 5) & (i.startswith('peak')):
                peak_list.append(h5file['shots'][j][i]['diffraction'][:])
                peak_list.append(h5file['shots'][j][i]['centerOfMassInPixels'][:])
    print('run %s: %d shots' % (run, n_shots))
    h5file.close()    
    return  {'peaks': peak_list,
             'wavelength': wavelength,
             'detector_distance': detector_distance,
             'center_mass': center_mass,
             'No_of_peaks': n_peaks}

In [57]:
sum(np.array(run_list['r0144']['No_of_peaks']))

433

In [56]:
run_list = {}
for n in files.keys():
    print("---------- %s ----------" % n)
    run_list[n] = load_runs(n)

---------- r0144 ----------
run r0144: 51 shots
---------- r0145 ----------
run r0145: 42 shots
---------- r0146 ----------
run r0146: 22 shots
---------- r0151 ----------
run r0151: 127 shots
---------- r0165 ----------
run r0165: 2865 shots
---------- r0166 ----------
run r0166: 630 shots
---------- r0167 ----------
run r0167: 786 shots
---------- r0168 ----------
run r0168: 625 shots
---------- r0169 ----------
run r0169: 499 shots
---------- r0170 ----------
run r0170: 140 shots
---------- r0171 ----------
run r0171: 433 shots
---------- r0172 ----------
run r0172: 749 shots
---------- r0173 ----------
run r0173: 1705 shots


In [44]:
type(run_list['r0144']['peaks'])

list

In [36]:
# print the center of mass of the peaks inside each shot the coordinates are given in the whole image 1760
for n, run in enumerate(run_list.keys()):
    print(np.shape(run_list[run]['peaks']))

(754,)
(640,)
(318,)
(1960,)
(33094,)
(6100,)
(6542,)
(5170,)
(6320,)
(1762,)
(5586,)
(9788,)
(14096,)


In [22]:
BG_level = 100
photon_threshold = 2000
glob_center = []
sphericity_of_peak = []
for n, run in enumerate(run_list.keys()):
    p = run_list[run]['peaks']
    c_mass = run_list[run]['peaks'][1]
    for p_i in np.arange(len(p)):
        p_sub = p[p_i] - BG_level
        p_mask = p_sub > photon_threshold
        fill_holes = binary_fill_holes(p_mask)
        contours,_= cv2.findContours(fill_holes.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        for cnt in contours:
            perimeter = cv2.arcLength(cnt, True)
            area = cv2.contourArea(cnt)
            if perimeter != 0:
                sphericity = 4*np.pi*area/(perimeter**2)     
                cnt_string = 'Numbers of contours for peak {} : {}, sphericity: {}'
                print(cnt_string.format(p_i, len(contours), sphericity))
                sphericity_of_peak.append((p[p_i],sphericity))

Numbers of contours for peak 2 : 1, sphericity: 0.5916196086304003
Numbers of contours for peak 4 : 1, sphericity: 0.7319212544653162
Numbers of contours for peak 6 : 2, sphericity: 0.8090594231490545
Numbers of contours for peak 6 : 2, sphericity: 0.7256046329210497
Numbers of contours for peak 8 : 1, sphericity: 0.7689393938672696
Numbers of contours for peak 10 : 1, sphericity: 0.3975089571665079
Numbers of contours for peak 12 : 2, sphericity: 0.0
Numbers of contours for peak 12 : 2, sphericity: 0.8575382908489096
Numbers of contours for peak 14 : 1, sphericity: 0.7255305594341376
Numbers of contours for peak 16 : 1, sphericity: 0.6981317007977318
Numbers of contours for peak 18 : 3, sphericity: 0.35447188368812665
Numbers of contours for peak 20 : 1, sphericity: 0.6091793466869331
Numbers of contours for peak 22 : 1, sphericity: 0.6817480478104881
Numbers of contours for peak 24 : 1, sphericity: 0.5598037438500115
Numbers of contours for peak 26 : 2, sphericity: 0.3224580264151723

Numbers of contours for peak 542 : 1, sphericity: 0.5638888941900405
Numbers of contours for peak 544 : 2, sphericity: 0.698733140133956
Numbers of contours for peak 544 : 2, sphericity: 0.7300079258206149
Numbers of contours for peak 546 : 3, sphericity: 0.5390120920947209
Numbers of contours for peak 546 : 3, sphericity: 0.5715045502581996
Numbers of contours for peak 548 : 2, sphericity: 0.6430287075658971
Numbers of contours for peak 548 : 2, sphericity: 0.6981317007977318
Numbers of contours for peak 550 : 1, sphericity: 0.6430287075658971
Numbers of contours for peak 552 : 1, sphericity: 0.668162124198742
Numbers of contours for peak 554 : 2, sphericity: 0.4836870396227584
Numbers of contours for peak 554 : 2, sphericity: 0.7867732359754227
Numbers of contours for peak 556 : 1, sphericity: 0.6960774634321742
Numbers of contours for peak 558 : 1, sphericity: 0.5417164797380755
Numbers of contours for peak 560 : 1, sphericity: 0.7354378163576873
Numbers of contours for peak 562 : 1

Numbers of contours for peak 360 : 1, sphericity: 0.37798726738580624
Numbers of contours for peak 362 : 2, sphericity: 0.0
Numbers of contours for peak 362 : 2, sphericity: 0.34290273015491973
Numbers of contours for peak 364 : 1, sphericity: 0.513012805217498
Numbers of contours for peak 366 : 1, sphericity: 0.7124045305189995
Numbers of contours for peak 368 : 1, sphericity: 0.34710380724364626
Numbers of contours for peak 372 : 1, sphericity: 0.6772362685323776
Numbers of contours for peak 374 : 5, sphericity: 0.6170722209589364
Numbers of contours for peak 374 : 5, sphericity: 0.4096491923977304
Numbers of contours for peak 374 : 5, sphericity: 0.4011640156922039
Numbers of contours for peak 374 : 5, sphericity: 0.0
Numbers of contours for peak 374 : 5, sphericity: 0.8061450759506535
Numbers of contours for peak 376 : 3, sphericity: 0.6545151309589601
Numbers of contours for peak 378 : 3, sphericity: 0.0
Numbers of contours for peak 378 : 3, sphericity: 0.7853981633974483
Numbers 

Numbers of contours for peak 298 : 1, sphericity: 0.38929644237788114
Numbers of contours for peak 300 : 2, sphericity: 0.5715045502581996
Numbers of contours for peak 302 : 2, sphericity: 0.30826198830475116
Numbers of contours for peak 302 : 2, sphericity: 0.772283996990162
Numbers of contours for peak 304 : 1, sphericity: 0.38928651324345726
Numbers of contours for peak 306 : 1, sphericity: 0.6305345791859771
Numbers of contours for peak 308 : 1, sphericity: 0.0
Numbers of contours for peak 312 : 1, sphericity: 0.41467149779294665
Numbers of contours for peak 314 : 1, sphericity: 0.601119055910142
Numbers of contours for peak 316 : 1, sphericity: 0.6213742116647135
Numbers of contours for peak 0 : 1, sphericity: 0.33310388024806986
Numbers of contours for peak 2 : 1, sphericity: 0.3914657533487695
Numbers of contours for peak 4 : 4, sphericity: 0.0
Numbers of contours for peak 4 : 4, sphericity: 0.6362380867588684
Numbers of contours for peak 4 : 4, sphericity: 0.485261057020188
Num

Numbers of contours for peak 564 : 3, sphericity: 0.22985234357421663
Numbers of contours for peak 564 : 3, sphericity: 0.276407510588395
Numbers of contours for peak 566 : 2, sphericity: 0.36775428324957027
Numbers of contours for peak 566 : 2, sphericity: 0.6545151309589601
Numbers of contours for peak 568 : 3, sphericity: 0.0
Numbers of contours for peak 568 : 3, sphericity: 0.16367120845388497
Numbers of contours for peak 570 : 1, sphericity: 0.3332077261131688
Numbers of contours for peak 572 : 1, sphericity: 0.4100590042717271
Numbers of contours for peak 574 : 1, sphericity: 0.7907055981667803
Numbers of contours for peak 576 : 1, sphericity: 0.8391855758365734
Numbers of contours for peak 578 : 1, sphericity: 0.3754251260321512
Numbers of contours for peak 580 : 1, sphericity: 0.6092120771088535
Numbers of contours for peak 582 : 1, sphericity: 0.717120852336662
Numbers of contours for peak 584 : 1, sphericity: 0.839909037392062
Numbers of contours for peak 586 : 1, sphericity:

...

Numbers of contours for peak 11212 : 2, sphericity: 0.31987786907807864
Numbers of contours for peak 11212 : 2, sphericity: 0.64735898263557
Numbers of contours for peak 11214 : 1, sphericity: 0.5468093062649478
Numbers of contours for peak 11216 : 2, sphericity: 0.7215119258605592
Numbers of contours for peak 11216 : 2, sphericity: 0.5390120920947209
Numbers of contours for peak 11218 : 1, sphericity: 0.7519551452394986
Numbers of contours for peak 11220 : 1, sphericity: 0.720566996684918
Numbers of contours for peak 11222 : 2, sphericity: 0.7154915583244541
Numbers of contours for peak 11222 : 2, sphericity: 0.5390120920947209
Numbers of contours for peak 11224 : 2, sphericity: 0.24197550056809783
Numbers of contours for peak 11226 : 1, sphericity: 0.750114377460519
Numbers of contours for peak 11228 : 1, sphericity: 0.660786705000382
Numbers of contours for peak 11230 : 1, sphericity: 0.6964605046056132
Numbers of contours for peak 11232 : 1, sphericity: 0.8927387775318816
Numbers o

Numbers of contours for peak 11866 : 1, sphericity: 0.9009979964885222
Numbers of contours for peak 11870 : 1, sphericity: 0.6872354633046636
Numbers of contours for peak 11872 : 2, sphericity: 0.5390120920947209
Numbers of contours for peak 11872 : 2, sphericity: 0.4611289111317869
Numbers of contours for peak 11876 : 1, sphericity: 0.8271345061569276
Numbers of contours for peak 11878 : 1, sphericity: 0.8927387775318816
Numbers of contours for peak 11880 : 2, sphericity: 0.6430287075658971
Numbers of contours for peak 11880 : 2, sphericity: 0.8001063703614795
Numbers of contours for peak 11882 : 2, sphericity: 0.8323186927986581
Numbers of contours for peak 11882 : 2, sphericity: 0.6981317007977318
Numbers of contours for peak 11886 : 1, sphericity: 0.299451162274845
Numbers of contours for peak 11888 : 1, sphericity: 0.5390120920947209
Numbers of contours for peak 11890 : 2, sphericity: 0.5390120920947209
Numbers of contours for peak 11890 : 2, sphericity: 0.5659309363890113
Numbers

Numbers of contours for peak 12500 : 1, sphericity: 0.5715045502581996
Numbers of contours for peak 12506 : 1, sphericity: 0.43833270788986844
Numbers of contours for peak 12508 : 1, sphericity: 0.4532305198179146
Numbers of contours for peak 12510 : 1, sphericity: 0.6987331381741835
Numbers of contours for peak 12512 : 2, sphericity: 0.38409806784026984
Numbers of contours for peak 12512 : 2, sphericity: 0.606388603606561
Numbers of contours for peak 12514 : 1, sphericity: 0.6232327314845211
Numbers of contours for peak 12516 : 1, sphericity: 0.6001223219087714
Numbers of contours for peak 12518 : 2, sphericity: 0.0
Numbers of contours for peak 12518 : 2, sphericity: 0.719225663292634
Numbers of contours for peak 12520 : 1, sphericity: 0.37100614245160407
Numbers of contours for peak 12522 : 1, sphericity: 0.41331844904212867
Numbers of contours for peak 12524 : 1, sphericity: 0.6619520067322555
Numbers of contours for peak 12526 : 1, sphericity: 0.5965201600509074
Numbers of contours

Numbers of contours for peak 13142 : 1, sphericity: 0.3826804489892827
Numbers of contours for peak 13144 : 2, sphericity: 0.0
Numbers of contours for peak 13144 : 2, sphericity: 0.38672633863047706
Numbers of contours for peak 13146 : 2, sphericity: 0.4641157057999475
Numbers of contours for peak 13148 : 1, sphericity: 0.5548791279733458
Numbers of contours for peak 13150 : 1, sphericity: 0.5262544094415914
Numbers of contours for peak 13152 : 1, sphericity: 0.6683749941974539
Numbers of contours for peak 13154 : 1, sphericity: 0.4646116503384779
Numbers of contours for peak 13156 : 2, sphericity: 0.7114959615650317
Numbers of contours for peak 13156 : 2, sphericity: 0.606388603606561
Numbers of contours for peak 13158 : 1, sphericity: 0.77435275056413
Numbers of contours for peak 13160 : 6, sphericity: 0.41920810543267806
Numbers of contours for peak 13160 : 6, sphericity: 0.0
Numbers of contours for peak 13160 : 6, sphericity: 0.2584276521084082
Numbers of contours for peak 13160 : 

Numbers of contours for peak 13796 : 3, sphericity: 0.8001063703614795
Numbers of contours for peak 13796 : 3, sphericity: 0.0
Numbers of contours for peak 13796 : 3, sphericity: 0.3799353255247749
Numbers of contours for peak 13798 : 2, sphericity: 0.6728516739410161
Numbers of contours for peak 13798 : 2, sphericity: 0.7392642964533103
Numbers of contours for peak 13802 : 2, sphericity: 0.8323186927986581
Numbers of contours for peak 13804 : 6, sphericity: 0.0
Numbers of contours for peak 13804 : 6, sphericity: 0.5390120920947209
Numbers of contours for peak 13804 : 6, sphericity: 0.34796955853223954
Numbers of contours for peak 13804 : 6, sphericity: 0.0
Numbers of contours for peak 13804 : 6, sphericity: 0.4342424731226482
Numbers of contours for peak 13806 : 1, sphericity: 0.5109475084176828
Numbers of contours for peak 13808 : 1, sphericity: 0.6667718224244321
Numbers of contours for peak 13810 : 2, sphericity: 0.41204480170999774
Numbers of contours for peak 13810 : 2, sphericit

In [5]:
# calculate qx and qy to find distance
wavelength = np.array([run_list[run]['wavelength'] for run in run_list.keys()])
detector_distance = np.array([run_list[run]['detector_distance'] for run in run_list.keys()])
pixel_size = 109.92*1e-6

center = None
x = np.arange(1760)
y = np.arange(1760)
xx, yy = np.meshgrid(x, y)
if center is None:
    xx = xx.astype(np.float64) - (1760-1)/2
    yy = yy.astype(np.float64) - (1760-1)/2
else:
    xx -= center[1]
    yy -= center[0]
rad = np.sqrt(xx*xx + yy*yy)
qx = []
qy = []
for i in range(len(wavelength)):
    qx.append(2.*np.pi*2.*np.sin(0.5*np.arctan2(xx*pixel_size, detector_distance[i]))/(wavelength[i]))
    qy.append(2.*np.pi*2.*np.sin(0.5*np.arctan2(yy*pixel_size, detector_distance[i]))/(wavelength[i]))

In [ ]:
# delta_cx = cx2 (old peak center in whole image 1760) - cx1 (old peak local center peak_region) # new_cx
# delta_cy = cy2 (old peak center in whole image 1760) - cy1 (old peak local center peak_region) # new_cx

In [ ]:
# new cx cut in whole image 1760 = new_cx(ROI) + delta_cx
# new cy cut in whole image 1760 = new_cy(ROI) + delta_cy

In [ ]:
delta_cx = run_list[run]['peaks'][1] - 

In [ ]:
delta_cy = 